# CGMacros Model Training and Evaluation

This notebook implements the complete machine learning pipeline for predicting Carbohydrate Caloric Ratio (CCR) from multimodal CGMacros data.

## Pipeline Overview
1. Data loading and preprocessing
2. Feature engineering
3. Model training (baseline, intermediate, advanced)
4. Model evaluation and comparison
5. Results analysis and visualization

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import warnings
from pathlib import Path

# Add src to path for imports
sys.path.append('../src')

from data_loader import load_cgmacros_data
from target import compute_ccr
from feature_engineering import engineer_features, get_feature_summary
from models import train_all_models, get_best_model
from evaluation import evaluate_models
from visualization import create_visualizations

# Settings
warnings.filterwarnings('ignore')
plt.style.use('default')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)

print("Libraries imported successfully!")
print("Ready to start the ML pipeline...")

## 1. Data Loading and Preprocessing

In [ ]:
# Load raw data
print("Loading CGMacros data...")
data_dir = "../data/raw"
raw_data = load_cgmacros_data(data_dir)

print(f"Raw data loaded: {raw_data.shape}")
print(f"Columns: {len(raw_data.columns)}")
print(f"Memory usage: {raw_data.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Display basic info
raw_data.info()

In [ ]:
# Compute target variable (CCR)
print("Computing Carbohydrate Caloric Ratio (CCR)...")
data_with_target = compute_ccr(raw_data)

print(f"Data after CCR computation: {data_with_target.shape}")

# Check if CCR was successfully computed
if 'ccr' in data_with_target.columns:
    ccr_stats = data_with_target['ccr'].describe()
    print("\nCCR Statistics:")
    print(ccr_stats)
    
    # Quick visualization of CCR distribution
    plt.figure(figsize=(10, 4))
    
    plt.subplot(1, 2, 1)
    plt.hist(data_with_target['ccr'].dropna(), bins=30, alpha=0.7, color='skyblue', edgecolor='black')
    plt.xlabel('CCR Value')
    plt.ylabel('Frequency')
    plt.title('CCR Distribution')
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1, 2, 2)
    plt.boxplot(data_with_target['ccr'].dropna())
    plt.ylabel('CCR Value')
    plt.title('CCR Box Plot')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("WARNING: CCR could not be computed. Check if nutrient data is available.")
    print("Available columns:", list(data_with_target.columns))

## 2. Feature Engineering

In [ ]:
# Perform feature engineering
print("Starting feature engineering pipeline...")
features_df = engineer_features(data_with_target)

print(f"Features after engineering: {features_df.shape}")

# Get feature summary
feature_summary = get_feature_summary(features_df)
print("\nFeature Engineering Summary:")
for key, value in feature_summary.items():
    print(f"{key}: {value}")

In [ ]:
# Analyze feature categories
feature_groups = feature_summary.get('feature_groups', {})

if feature_groups:
    # Visualize feature distribution by category
    plt.figure(figsize=(12, 6))
    
    categories = list(feature_groups.keys())
    counts = list(feature_groups.values())
    
    plt.bar(categories, counts, alpha=0.7, color='lightcoral')
    plt.xlabel('Feature Category')
    plt.ylabel('Number of Features')
    plt.title('Feature Distribution by Category')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3, axis='y')
    
    # Add count labels on bars
    for i, count in enumerate(counts):
        plt.text(i, count + 0.5, str(count), ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print("\nFeature counts by category:")
    for category, count in feature_groups.items():
        print(f"  {category}: {count} features")

In [ ]:
# Check data quality after feature engineering
print("Data quality assessment after feature engineering:")

# Missing data
missing_pct = (features_df.isnull().sum() / len(features_df)) * 100
high_missing = missing_pct[missing_pct > 10].sort_values(ascending=False)

if len(high_missing) > 0:
    print(f"\nFeatures with >10% missing data: {len(high_missing)}")
    print(high_missing.head(10))
else:
    print("\nNo features with >10% missing data - excellent!")

# Data types
print(f"\nData types:")
print(features_df.dtypes.value_counts())

# Memory usage
memory_mb = features_df.memory_usage(deep=True).sum() / 1024**2
print(f"\nMemory usage: {memory_mb:.2f} MB")

## 3. Model Training

In [ ]:
# Train all models
print("Starting model training pipeline...")
print("This may take several minutes depending on data size...")

# Check if we have the target variable
if 'ccr' not in features_df.columns:
    print("ERROR: CCR target variable not found in features DataFrame")
    print("Available columns:", list(features_df.columns))
else:
    # Train models
    models_results = train_all_models(features_df, save_dir='../models')
    
    print("\nModel training completed!")
    print(f"Models trained: {len(models_results.get('models', {}))}")
    
    # Display training results
    evaluation_results = models_results.get('evaluation_results', {})
    
    if evaluation_results:
        print("\nModel Performance Summary:")
        print("-" * 60)
        print(f"{'Model':<20} {'NRMSE':<10} {'Correlation':<12} {'R²':<10}")
        print("-" * 60)
        
        for model_name, metrics in evaluation_results.items():
            nrmse = metrics.get('nrmse', 'N/A')
            corr = metrics.get('correlation', 'N/A')
            r2 = metrics.get('r2', 'N/A')
            
            nrmse_str = f"{nrmse:.4f}" if isinstance(nrmse, (int, float)) else str(nrmse)
            corr_str = f"{corr:.4f}" if isinstance(corr, (int, float)) else str(corr)
            r2_str = f"{r2:.4f}" if isinstance(r2, (int, float)) else str(r2)
            
            print(f"{model_name:<20} {nrmse_str:<10} {corr_str:<12} {r2_str:<10}")
        
        # Find best model
        best_model, best_score = get_best_model(evaluation_results, 'nrmse')
        if best_model:
            print(f"\nBest Model: {best_model} (NRMSE: {best_score:.4f})")

## 4. Model Evaluation and Analysis

In [ ]:
# Comprehensive model evaluation
if 'models_results' in locals() and models_results:
    print("Performing comprehensive model evaluation...")
    
    comprehensive_results = evaluate_models(models_results, features_df)
    
    print("Evaluation completed!")
    
    # Display evaluation summary
    eval_summary = comprehensive_results.get('evaluation_summary', {})
    
    if eval_summary:
        print("\nEvaluation Summary:")
        print(f"Total models evaluated: {eval_summary.get('total_models', 'N/A')}")
        
        best_model_info = eval_summary.get('best_overall_model', {})
        if best_model_info:
            print(f"Best overall model: {best_model_info.get('name', 'N/A')}")
            print(f"Best NRMSE: {best_model_info.get('nrmse', 'N/A')}")
            print(f"Best Correlation: {best_model_info.get('correlation', 'N/A')}")
        
        key_findings = eval_summary.get('key_findings', [])
        if key_findings:
            print("\nKey Findings:")
            for finding in key_findings:
                print(f"  • {finding}")
else:
    print("Models not available for evaluation. Please run the training cell first.")

In [ ]:
# Model comparison visualization
if 'comprehensive_results' in locals():
    model_metrics = comprehensive_results.get('model_metrics', {})
    
    if model_metrics:
        # Create performance comparison plot
        metrics_df = pd.DataFrame(model_metrics).T
        
        if not metrics_df.empty:
            fig, axes = plt.subplots(2, 2, figsize=(15, 10))
            fig.suptitle('Model Performance Comparison', fontsize=16, fontweight='bold')
            
            # NRMSE comparison
            if 'nrmse' in metrics_df.columns:
                axes[0, 0].bar(metrics_df.index, metrics_df['nrmse'], color='lightcoral', alpha=0.8)
                axes[0, 0].set_title('NRMSE (Lower is Better)')
                axes[0, 0].set_ylabel('NRMSE')
                axes[0, 0].tick_params(axis='x', rotation=45)
                axes[0, 0].grid(True, alpha=0.3, axis='y')
            
            # Correlation comparison
            if 'correlation' in metrics_df.columns:
                axes[0, 1].bar(metrics_df.index, metrics_df['correlation'], color='lightgreen', alpha=0.8)
                axes[0, 1].set_title('Correlation (Higher is Better)')
                axes[0, 1].set_ylabel('Correlation')
                axes[0, 1].tick_params(axis='x', rotation=45)
                axes[0, 1].grid(True, alpha=0.3, axis='y')
            
            # R² comparison
            if 'r2' in metrics_df.columns:
                axes[1, 0].bar(metrics_df.index, metrics_df['r2'], color='lightblue', alpha=0.8)
                axes[1, 0].set_title('R² Score (Higher is Better)')
                axes[1, 0].set_ylabel('R²')
                axes[1, 0].tick_params(axis='x', rotation=45)
                axes[1, 0].grid(True, alpha=0.3, axis='y')
            
            # MAE comparison
            if 'mae' in metrics_df.columns:
                axes[1, 1].bar(metrics_df.index, metrics_df['mae'], color='plum', alpha=0.8)
                axes[1, 1].set_title('MAE (Lower is Better)')
                axes[1, 1].set_ylabel('MAE')
                axes[1, 1].tick_params(axis='x', rotation=45)
                axes[1, 1].grid(True, alpha=0.3, axis='y')
            
            plt.tight_layout()
            plt.show()

## 5. Feature Importance Analysis

In [ ]:
# Feature importance analysis
if 'comprehensive_results' in locals():
    feature_importance = comprehensive_results.get('feature_importance', {})
    
    if feature_importance:
        top_features = feature_importance.get('top_features', {})
        
        if 'mean_importance' in top_features:
            # Plot top important features
            plt.figure(figsize=(12, 8))
            
            # Sort features by importance
            importance_items = list(top_features['mean_importance'].items())
            importance_items.sort(key=lambda x: x[1], reverse=True)
            
            # Take top 15 features
            top_15 = importance_items[:15]
            features, importances = zip(*top_15)
            
            plt.barh(range(len(features)), importances, color='steelblue', alpha=0.8)
            plt.yticks(range(len(features)), features)
            plt.xlabel('Mean Feature Importance')
            plt.title('Top 15 Most Important Features', fontsize=14, fontweight='bold')
            plt.grid(True, alpha=0.3, axis='x')
            
            plt.tight_layout()
            plt.show()
            
            print("Top 10 Most Important Features:")
            for i, (feature, importance) in enumerate(top_15[:10], 1):
                print(f"{i:2d}. {feature:<30} {importance:.4f}")
        
        # Category importance
        category_importance = feature_importance.get('category_importance', {})
        
        if category_importance:
            print("\nFeature Importance by Category:")
            print("-" * 50)
            
            for category, info in category_importance.items():
                mean_imp = info.get('mean_importance', 0)
                feature_count = info.get('feature_count', 0)
                top_feature = info.get('top_feature', 'N/A')
                
                print(f"{category:<15} Mean: {mean_imp:.4f}, Count: {feature_count:2d}, Top: {top_feature}")
    else:
        print("Feature importance data not available")
else:
    print("Evaluation results not available. Please run the evaluation cell first.")

## 6. Prediction Analysis

In [ ]:
# Prediction vs actual analysis for best model
if 'models_results' in locals() and 'comprehensive_results' in locals():
    trainer = models_results.get('trainer')
    data_splits = models_results.get('data_splits', {})
    
    X_test = data_splits.get('X_test')
    y_test = data_splits.get('y_test')
    
    if trainer and X_test is not None and y_test is not None:
        # Get best model
        eval_summary = comprehensive_results.get('evaluation_summary', {})
        best_model_info = eval_summary.get('best_overall_model', {})
        best_model_name = best_model_info.get('name')
        
        if best_model_name and best_model_name in trainer.models:
            # Make predictions
            y_pred = trainer.predict(best_model_name, X_test)
            
            # Create prediction analysis plot
            fig, axes = plt.subplots(1, 3, figsize=(18, 5))
            fig.suptitle(f'Prediction Analysis - {best_model_name.replace("_", " ").title()}', 
                        fontsize=16, fontweight='bold')
            
            # Predicted vs Actual scatter plot
            axes[0].scatter(y_test, y_pred, alpha=0.6, s=50)
            min_val = min(y_test.min(), y_pred.min())
            max_val = max(y_test.max(), y_pred.max())
            axes[0].plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect Prediction')
            axes[0].set_xlabel('Actual CCR')
            axes[0].set_ylabel('Predicted CCR')
            axes[0].set_title('Predicted vs Actual')
            axes[0].grid(True, alpha=0.3)
            axes[0].legend()
            
            # Residuals plot
            residuals = y_test - y_pred
            axes[1].scatter(y_pred, residuals, alpha=0.6)
            axes[1].axhline(y=0, color='red', linestyle='--', linewidth=2)
            axes[1].set_xlabel('Predicted CCR')
            axes[1].set_ylabel('Residuals')
            axes[1].set_title('Residuals vs Predicted')
            axes[1].grid(True, alpha=0.3)
            
            # Residuals histogram
            axes[2].hist(residuals, bins=20, alpha=0.7, color='skyblue', edgecolor='black')
            axes[2].axvline(x=0, color='red', linestyle='--', linewidth=2)
            axes[2].set_xlabel('Residuals')
            axes[2].set_ylabel('Frequency')
            axes[2].set_title('Residuals Distribution')
            axes[2].grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.show()
            
            # Prediction statistics
            mse = np.mean(residuals**2)
            rmse = np.sqrt(mse)
            mae = np.mean(np.abs(residuals))
            r2 = 1 - np.sum(residuals**2) / np.sum((y_test - y_test.mean())**2)
            correlation = np.corrcoef(y_test, y_pred)[0, 1]
            
            print(f"\nPrediction Statistics for {best_model_name}:")
            print(f"RMSE: {rmse:.4f}")
            print(f"MAE: {mae:.4f}")
            print(f"R²: {r2:.4f}")
            print(f"Correlation: {correlation:.4f}")
            print(f"Mean Residual: {np.mean(residuals):.4f}")
            print(f"Std Residual: {np.std(residuals):.4f}")
        else:
            print("Best model not found or not available for prediction")
    else:
        print("Test data not available for prediction analysis")
else:
    print("Model results not available. Please run the training cells first.")

## 7. Save Results and Generate Report

In [ ]:
# Save processed data
if 'features_df' in locals():
    output_dir = Path('../data/processed')
    output_dir.mkdir(exist_ok=True)
    
    features_file = output_dir / 'features_with_target.csv'
    features_df.to_csv(features_file, index=False)
    print(f"Processed features saved to: {features_file}")

# Generate visualizations
if 'models_results' in locals() and 'comprehensive_results' in locals():
    print("\nGenerating comprehensive visualizations...")
    
    results_dir = Path('../results')
    results_dir.mkdir(exist_ok=True)
    
    try:
        create_visualizations(
            features_df, 
            models_results, 
            comprehensive_results, 
            str(results_dir)
        )
        print(f"Visualizations saved to: {results_dir}")
    except Exception as e:
        print(f"Error generating visualizations: {str(e)}")

# Summary of outputs
print("\n" + "="*60)
print("PIPELINE EXECUTION SUMMARY")
print("="*60)

if 'features_df' in locals():
    print(f"✓ Feature engineering completed: {features_df.shape[1]} features")

if 'models_results' in locals():
    num_models = len(models_results.get('models', {}))
    print(f"✓ Model training completed: {num_models} models trained")

if 'comprehensive_results' in locals():
    print(f"✓ Model evaluation completed")
    
    eval_summary = comprehensive_results.get('evaluation_summary', {})
    best_model_info = eval_summary.get('best_overall_model', {})
    
    if best_model_info:
        best_name = best_model_info.get('name', 'Unknown')
        best_nrmse = best_model_info.get('nrmse', 'N/A')
        best_corr = best_model_info.get('correlation', 'N/A')
        
        print(f"✓ Best model: {best_name}")
        print(f"  - NRMSE: {best_nrmse}")
        print(f"  - Correlation: {best_corr}")

print(f"✓ Results saved to: ../results/")
print(f"✓ Models saved to: ../models/")
print(f"✓ Processed data saved to: ../data/processed/")

print("\n" + "="*60)
print("Pipeline execution completed successfully!")
print("Check the results directory for detailed outputs.")
print("="*60)